## 1: Loading Dependencies, Packages and Data

In [32]:
# Core Dependencies
import re
import os

from eda import EDA

# Start Python Imports
import math, time, random, datetime

# Data Manipulation
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
pd.options.display.min_rows = 200

# Visualization 
import matplotlib.pyplot as plt
import missingno
%matplotlib inline 
import seaborn as sns
plt.style.use('seaborn-whitegrid')

import warnings
warnings.filterwarnings("ignore")

In [33]:
accepted_file = "../input/lending-club/accepted_2007_to_2018Q4.csv.gz"
rejected_file = "../input/lending-club/rejected_2007_to_2018Q4.csv.gz"

In [34]:
acc_df = pd.read_csv(accepted_file)
rej_df = pd.read_csv(rejected_file)

In [35]:
print(acc_df.shape)
print(rej_df.shape)

## 2: Feature Selection

Due to the large volume of data in the `accepted_2007_to_2018Q4.csv.gz`, of aroud - **(2260701, 151)**. I would be selectively choosing the columns based on the relevancy of the assignment

In [36]:
columns = ["loan_amnt", "term", "int_rate", "installment", "grade", "sub_grade", "emp_title",
           "emp_length", "home_ownership", "annual_inc", "verification_status", "issue_d", "loan_status",
           "purpose", "title", "dti", "earliest_cr_line", "open_acc", "pub_rec", "revol_bal", "revol_util",
           "total_acc", "initial_list_status", "application_type", "mort_acc", "pub_rec_bankruptcies", "address"]

Finding the intersection between the above columns and the dataframe columns (used for selection) and also excluding columns further which have atleast 20% NaN values.

In [37]:
acc_df = acc_df[acc_df.columns.intersection(columns)]

perc = 20.0 
min_count =  int(((100-perc)/100)*acc_df.shape[0] + 1)
acc_df = acc_df.dropna( axis=1, 
                thresh=min_count)

acc_df = acc_df.sample(frac=0.5, replace=True, random_state=1)

In [38]:
# New shape of the dataframe
acc_df.shape

Birds eye view of the both the datasets

In [39]:
acc_df.describe()

In [40]:
rej_df.describe()

In [41]:
acc_df = acc_df.loc[~acc_df.index.duplicated(), :]

## 3: Visualization Plots

### 3.0 Helper Functions


In [42]:
def missing_value_counter(df):
    percent_missing_values = df.isnull().sum()
    percent_missing_percentage = df.isnull().sum() * 100 / len(df)
    
    missing_value_df = pd.DataFrame({'column_name': df.columns,
                                     'percent_missing': percent_missing_percentage,
                                    'count_missing': percent_missing_values})
    missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)
    return missing_value_df.head(10)

### 3.1 Missing Value Visualization

The plot below gives an overview of the columnes and their missing data.

In [43]:
EDA.plot1(acc_df)

In [44]:
missing_value_counter(acc_df)

The above is a count and percentage of the top 10 missing values of the accpeted data

## 3.2 Loan Status Count

This plot show the count of account based on their loan status

In [46]:
EDA.plot2(acc_df)

Fully paid are the highest followed by current and the charged off. Most of the loan accounts are either paid off or the payment is still in progress

## 3.3 HeatMap

The below heatmap shows a correlation between attributes. A few key observations from the heatmap is that:
- installment and loan amount are heavily correlated, which implies greater the loan amount, higher the installments and vice versa.
- total_acc and open_acc are correlated
- publci_rec_bankruptcies and public_rec are correlated

In [47]:
EDA.plot3(acc_df)

## 3.4 Grade Count Plot

A higher LC assigned loan grade implies the more likely a borrower would repay the loan. The acceptance rate is more distributed to candidates with a grade of `A, B and C`. 

In [48]:
EDA.plot4(acc_df)

## 3.5 Installment vs Loan Status vs Count

In [50]:
EDA.plot5(acc_df)

## 3.6 Loan Status vs Other Attributes

The below plots have 4 subplots where each certain significance <br>
- **Plot 1:** Terms vs Loan Status - Relation between how terms affected the loan status. 
- **Plot 2:** Home Onwership vs Loan Status - No significant differnece between this comparision
- **Plot 3:** Verification Status vs Loan Status 
- **Plot 4:** Debt vs Loan Status - Loan charged off significantly for debt consolidation



In [51]:
paid_charged = acc_df.loc[(acc_df["loan_status"] == "Fully Paid") | (acc_df["loan_status"] == "Charged Off")]
EDA.plot6(paid_charged)

## 3.7 Grade vs Loan Status
Key Observations:
- As grade goes higher the count of charged off increases. There is a higher risk in allocating loan to candidates with a higher grade.
- Grade C onwards has higher loan fully paid to charged off ratio.
- Looking at the sub grade, it follows a normal distribution of loan fully paid to charged off ratio.

In [52]:
EDA.plot7(paid_charged)

## 3.8 Employment vs Loan Status
- Candidates with over 10+ years of experience tend to have lower fully paid to charged off ratio

In [ ]:
EDA.plot8(paid_charged)

## 3.9 Employee Title vs Loans Accepted
- Maximum number of loans are accepted for professions with a more stable jobs (according to the society) like Teacher, Nurse, Owner and etc..

In [53]:
EDA.plot9(paid_charged)

## 3.10 Application Type vs Loan Status
- This plot focuses more on loan status in accordance with the application type

In [54]:
EDA.plot10(paid_charged)